In [1]:
# @title Import necessary modules
import pandas as pd
import time
from datetime import datetime
import numpy as np
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient
from bokeh.io import (output_notebook,
                      show,
                      push_notebook,
                      curdoc,
                      )
from bokeh.models import (ColumnDataSource,
                          DatetimeTickFormatter,
                          NumeralTickFormatter,
                          HoverTool)
from bokeh.plotting import figure

In [2]:
# @title
client = InfluxDBClient(url='http://epics.ipfn.tecnico.ulisboa.pt:8086',
                        token='kJXa2Ok88yE8nSEyh2lStvs6v3vWYGhb90XL3M-gGhuysJWj36XaJkQaXSnGUa7h6FdkiIhZJ3JAYVCpc0390w==',
                        org='67c974ed52348dd6')
query_api = client.query_api()
wExp = widgets.Dropdown(
    options=['calorimetry', 'calorimetryEsp32', ],
    value='calorimetry',
    description='Experiment:',
    disabled=False,
)
display(wExp)

Dropdown(description='Experiment:', options=('calorimetry', 'calorimetryEsp32'), value='calorimetry')

# Calculations

In [3]:
# @title Get saved Data
DATE  = '2024-12-10'
START = 'T23:20:00'
STOP  = 'T23:59:00'

RANGE = f'start:{DATE:s}{START:s}Z, stop:{DATE:s}{STOP:s}Z'
output_notebook()
query_saved_temp = (f'from(bucket:"ardu-rasp") |> range({RANGE}) '
    f'|> filter(fn: (r) => r.experiment == "{wExp.value}") '
    '|> filter(fn: (r) => r._measurement == "ds18b20Sensors") '
    '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
query_api = client.query_api()
# Bokeh Time formatter
ft2 = DatetimeTickFormatter(
    milliseconds = "%H:%M:%S",
    seconds = "%H:%M:%S",
    minutes = "%H:%M:%S"
)
temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_saved_temp)
source = ColumnDataSource(temperature_table_df)
TOOLTIPS = [
    ("index", "$index"),
    ("Temp", "$y ºC"),
]

fig = figure(title="ARDUINO-RASP Temperature Plot",
            x_axis_type="datetime",
            sizing_mode="stretch_width",
            x_axis_label='Time',
            y_axis_label='ºC',
            y_range=(15, 100),
            tooltips=TOOLTIPS,)

fig.line(x='_time', y='tempBlue', source=source, color="blue", line_width=2)
fig.line(x='_time', y='tempRed', source=source, color="red", line_width=2)
fig.xaxis[0].formatter = ft2
t=show(fig, notebook_handle=True)

Loading BokehJS ...

In [5]:
tRed = temperature_table_df['tempRed']
tBlue = temperature_table_df['tempBlue']
# @title Save CSV File (You may need to download file from 'Files' section)
T_mat = np.array([tRed, tBlue])
filename = f"Temperature_log_{DATE:s}{START:s}.csv"
np.savetxt(filename, T_mat.T, delimiter=",")

In [6]:
# 1 Calorie
# select interval index
start_iloc = 114
end_iloc = 286

cH2O = 4.184  # J
# MassCopper = 235 g
Mcu = 235

# MCal + Cover 
MCal_Cover = 320 # g
# Mcal + Cover + H20 
Mcal_Cover_H2O = 475
MH2O = Mcal_Cover_H2O - MCal_Cover 

Tinitial = tRed.iloc[start_iloc]
THot = tBlue.iloc[start_iloc]
Tequil = tRed.iloc[end_iloc]

MH2O, Tinitial, THot, Tequil
# cH2O * MH2O * (Tequil - Tinitial) = cCu * Mcu * (THot - Tequil)
cCu = cH2O * MH2O * (Tequil - Tinitial) / ( Mcu * (THot - Tequil))
print(f"Mass H2O: {MH2O}, Mass Cu: {Mcu}")
print(f"T initial: {Tinitial:0.2f} ºC, THot: {THot:0.2f} ºC, Tequil: {Tequil:0.2f} ºC")
print(f"cCu: {cCu:0.3f} J/(g K)")

Mass H2O: 155, Mass Cu: 235
T initial: 21.44 ºC, THot: 80.31 ºC, Tequil: 28.69 ºC
cCu: 0.388 J/(g K)


In [7]:
# @title Calculations for Aluminium
# select interval index
start_iloc = 1113
end_iloc = 1456

cH2O = 4.184

# Aluminium = 57 g
mAl = 57

# MCal + Cover = 320 g
mCal_Cover = 320 # g
# Mcal + Cover + H20 = 481
mCal_Cover_H2O = 481
mH2O = mCal_Cover_H2O - mCal_Cover
Tinitial = tRed.iloc[start_iloc]
THot = tBlue.iloc[start_iloc]
Tequil = tRed.iloc[end_iloc]

mH2O, Tinitial, THot, Tequil
# cH2O * MH2O * (Tequil - Tinitial) = cCu * Mcu * (THot - Tequil)
cAl = cH2O * mH2O * (Tequil - Tinitial) / ( mAl * (THot - Tequil))
print(f"Mass H2O: {mH2O}, Mass Al: {mAl}")
print(f"T initial: {Tinitial}, THot: {THot}, Tequil: {Tequil}")
print(f"cAl: {cAl:0.3f} J/(g K)")

Mass H2O: 161, Mass Al: 57
T initial: 21.625, THot: 87.8125, Tequil: 26.1875
cAl: 0.875 J/(g K)


In [ ]:
#import influxdb_client
#print("Influxdb_client Version : {}".format(influxdb_client.__version__))
#Influxdb_client Version : 1.47.0
#import bokeh
#print("Bokeh Version : {}".format(bokeh.__version__))
#Bokeh Version : 3.4.3